In [ ]:
from sentence_transformers import SentenceTransformer
from matplotlib import pyplot as plt
import pandas as pd
import sys
import os 


In [ ]:
sys.path.insert(0,'/home/salaris/protein_model/')
from functions import load_model,predict_sequence


In [ ]:
base_model = "/home/salaris/protein_model/function_prediction/protein_model/model/esm2_t6_8M_UR50D_base_model_accelerated/"
# finetuned_model = "/home/salaris/protein_model/function_prediction/protein_model/model/esm_finetuned_accelerated/deep-totem-10/"
# finetuned_model = "/home/salaris/protein_model/function_prediction/protein_model/model/esm_finetuned_accelerated/volcanic-cherry-11/"
# finetuned_model = "/home/salaris/protein_model/function_prediction/protein_model/model/esm_finetuned_accelerated/snowy-snowflake-13/"
# finetuned_model = "/home/salaris/protein_model/function_prediction/protein_model/model/esm_finetuned_accelerated/sleek-tree-17/"
finetuned_model = "/home/salaris/protein_model/function_prediction/protein_model/model/esm_finetuned_accelerated/still-tree-30/"

In [ ]:
from functions import EmbeddingEncoder

model,tokenizer = load_model(base_model,peft_model_path=finetuned_model,only_raw_model=False)

modelname = 'tavakolih/all-MiniLM-L6-v2-pubmed-full'
# model = SentenceTransformer(modelname)


In [ ]:
st_model = EmbeddingEncoder(modelname)
emb =st_model.encode('hi')

In [ ]:
sequence = ["MKMKIEFSTGCIPISSLFMSIIKEKEKNNKKTKNFTFSVYIKKYSIEGDNFIIEDKVILNISTPDLELGFHIYNGLMTS"]
xx,yy = predict_sequence(model,tokenizer,sequence,max_length=1024,to_cuda=False)

In [ ]:
output_file = '/home/salaris/protein_model/function_prediction/data/prot_func.tsv'

all_data = pd.read_csv(output_file,sep = "\t", nrows = 10000)


In [ ]:
#lets pick something:
candidate = all_data.iloc[0]
len(candidate.sequence)


In [ ]:
candidate

In [ ]:
candidate_embedding = st_model.encode(candidate.go)
# print(candidate_embedding.shape)
candidate_embedding

In [ ]:
# ! pip install deap
# !pip install Levenshtein
xx = predict_sequence(model, tokenizer, sequence,max_length=512,to_cuda=False)


In [ ]:
from deap import base, creator, tools, algorithms
import random
import numpy as np
from Levenshtein import distance
from sklearn.metrics import r2_score
sequence_length = 305
# Your specific starting sequence and positions to mutate

amino_acids = 'ACDEFGHIKLMNPQRSTVWY'

# Generate a random sequence of 100 amino acids
# starting_sequence = ''.join(random.choice(amino_acids) for _ in range(sequence_length))
starting_sequence = candidate.sequence
print('starting sequence length:', len(starting_sequence))

positions_to_mutate = range(0,305,10)  # example positions
# positions_to_mutate = range(sequence_length)
# mutate some random positions 
ss = list(starting_sequence)
for position in positions_to_mutate:
    ss[position] = random.choice(amino_acids)
starting_sequence = "".join(ss)
print(starting_sequence)



# Define the fitness function
def fitness_function(sequence,model,tokenizer,candidate_embedding):
    # print(sequence)
    xx,yy = predict_sequence(model,tokenizer, "".join(sequence), max_length = 512, to_cuda= False)
    # print(xx.shape, candidate_embedding.shape)
    return(r2_score(xx.cpu().numpy().reshape(-1),candidate_embedding),)
# Define the sequence creator
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)



# Initialization of the population
toolbox = base.Toolbox()

# Define a generator function for new individuals
def generate_individual():
    return creator.Individual(list(starting_sequence))

toolbox.register("individual", generate_individual)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

# Custom mutation function
def custom_mutate(individual):
    for position in positions_to_mutate:
        # Randomly select a new amino acid for the position
        individual[position] = random.choice('ACDEFGHIKLMNPQRSTVWY')
    return individual,

# Operators
toolbox.register("evaluate", fitness_function,**{'model':model, 'tokenizer':tokenizer, 'candidate_embedding': candidate_embedding})
toolbox.register("mate", tools.cxUniform, indpb=0.1)  # Uniform crossover
toolbox.register("mutate", custom_mutate)
toolbox.register("select", tools.selTournament, tournsize=3)

# Genetic Algorithm
def main():
    pop0 = toolbox.population(n=50)  # n is the number of individuals
    hof = tools.HallOfFame(1, similar=lambda x, y: x == y)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("min", np.min)
    stats.register("max", np.max)

    pop, log = algorithms.eaSimple(pop0, toolbox, cxpb=0.5, mutpb=0.2, ngen=380, stats=stats, halloffame=hof, verbose=True)  # ngen is the number of generations

    return pop, log, hof,pop0,stats

# if __name__ == "__main__":
#     main()


In [ ]:
pop , log , hof,pop0,stats = main()

In [ ]:
np.array(pop).shape

In [ ]:
a = starting_sequence
a

In [ ]:
best ="".join(hof.items[0])

In [ ]:
ss =  candidate.sequence
ss

In [ ]:
distance(ss,best)

In [ ]:
distance(ss,starting_sequence)

In [ ]:
pop0 = toolbox.population(n=300)
a = "".join(pop0[0])
a

In [ ]:
max_r2 = r2_sc

In [ ]:
star